In [26]:
train_root = '/raid/binod/prashant/CVS_TRAIN'

import cv2
from PIL import Image

def extract_frame_by_index(video_path, frame_index, output_image_path=""):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    
    # Read the frame
    ret, frame = cap.read()
    
    if ret:
        # # Save the frame as an image
        # cv2.imwrite(output_image_path, frame)
        # print(f"Frame {frame_index} extracted and saved to {output_image_path}")
        pass
    else:
        print(f"Error: Could not read frame {frame_index}")
    
    # Release the video capture object
    cap.release()
    img = cv2.cvtColor(cv2.resize(frame, (256, 256)), cv2.COLOR_BGR2RGB)
    # cv2.imwrite(f'test_{frame_index}.jpg', frame)

    return img

def extract_prev_frames_by_index(video_path, frame_index, output_image_path=""):
    # Open the video file
    imgs = []
    for _ in range(5):
        cap = cv2.VideoCapture(video_path)
        
        # Check if the video opened successfully
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return
        
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        
        # Read the frame
        ret, frame = cap.read()
        
        if ret:
            # # Save the frame as an image
            # cv2.imwrite(output_image_path, frame)
            # print(f"Frame {frame_index} extracted and saved to {output_image_path}")
            pass
        else:
            print(f"Error: Could not read frame {frame_index}")
        
        # Release the video capture object
        cap.release()
        img = cv2.cvtColor(cv2.resize(frame, (256, 256)), cv2.COLOR_BGR2RGB)
        # cv2.imwrite(f'test_{frame_index}.jpg', frame)
        imgs.append(img)
        frame_index -= 30
        if frame_index < 0:
            break
    return imgs




In [27]:
import numpy as np
def find_mode(numbers_array):
    values, counts = np.unique(numbers_array, return_counts=True)
    
    # Find the index of the maximum count
    max_count_index = np.argmax(counts)
    
    # The mode is the value at the index of the maximum count
    mode = values[max_count_index]
    
    return mode


In [28]:
import os
import pandas as pd
import pickle
from tqdm import tqdm
def preprocess(videos_list, split, mode='train'):
    all_lists = []
    # for video_id_mp4 in tqdm(os.listdir(os.path.join(train_root, 'videos'))):
    for video_id in tqdm(videos_list):
        # video_id = video_id_mp4[:-4]
        if 'video_list' == '' or 'video_list' == None:
            print("empty, or none found")
            continue
        label_csv_path = os.path.join(train_root, 'labels', video_id, 'frame.csv')
        if label_csv_path == '/raid/binod/prashant/CVS_TRAIN/labels/frame.csv':
            print("error,", video_id)
            continue
        df  = pd.read_csv(label_csv_path)
        for index, row in df.iterrows():
            frame_index = row['frame_id']
            if mode == "val":
                c1 = find_mode(np.array([row['c1_rater1'], row['c1_rater2'], row['c1_rater3']]))
                c2 = find_mode(np.array([row['c2_rater1'], row['c2_rater2'], row['c2_rater3']]))
                c3 = find_mode(np.array([row['c3_rater1'], row['c3_rater2'], row['c3_rater3']]))
            elif mode == 'train':
                c1 = np.mean(np.array([row['c1_rater1'], row['c1_rater2'], row['c1_rater3']]))
                c2 = np.mean(np.array([row['c2_rater1'], row['c2_rater2'], row['c2_rater3']]))
                c3 = np.mean(np.array([row['c3_rater1'], row['c3_rater2'], row['c3_rater3']]))
            else:
                assert 1 == 0
            video_path = os.path.join(train_root, 'videos', f'{video_id}.mp4')

            
            img_array = extract_frame_by_index(video_path, frame_index)

            dictionary = {
                            'video_id': video_id,
                            'frame_id': frame_index,
                            'img_array': img_array,
                            'label': [c1, c2, c3]
                        }
            all_lists.append(dictionary)

    with open(f'single_images_mean_consistency_{split}.pkl', 'wb') as handle:
    # with open('multiple_images_1fps.pkl', 'wb') as handle:
        pickle.dump(all_lists, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [60]:
# opening the file in read mode
import random 
my_file = open("/raid/binod/prashant/videos_list.txt", "r") 
  
# reading the file 
data = my_file.read() 
  
# replacing end of line('/n') with ' ' and 
# splitting the text it further when '.' is seen. 
data_into_list = data.replace('\n', ' ').split(" ") 
  
# printing the data 
my_file.close() 

random.shuffle(data_into_list)
train_list = data_into_list[:int(0.8*len(data_into_list))]
val_list = data_into_list[int(0.8*len(data_into_list)):]
# print(train_list)
# print(val_list)
# preprocess(train_list, 'train', mode='train')
# preprocess(val_list, 'val', mode='val')

In [64]:
import cv2
import os
import pandas as pd
import csv
import json 

# Create the output directory if it doesn't exist
output_dir_train = 'output_frames_train'
if not os.path.exists(output_dir_train):
    os.makedirs(output_dir_train)

output_dir_val = 'output_frames_val'
if not os.path.exists(output_dir_val):
    os.makedirs(output_dir_val)

label_dict = {}


for filename in tqdm(train_list):

    # Set the path to the video file and CSV file
    filename = filename.split('.')[0]
    video_path = os.path.join(train_root, f'videos/{filename}.mp4')
    csv_path = os.path.join(train_root, f'labels/{filename}/frame.csv')

    # Open the video capture
    cap = cv2.VideoCapture(video_path)

    # Get the video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Read the CSV file into a pandas DataFrame
    labels_df = pd.read_csv(csv_path)

    # Initialize the frame counter
    frame_counter = 0


    # # Loop through the frames
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the frame was read successfully
        if not ret:
            break

        # Get the labels for the current frame
        # print(frame_counter)
        if (frame_counter==0 or frame_counter%150 == 0):
            # print(frame_counter)
            label = labels_df.loc[labels_df['frame_id'] == frame_counter].values
            # print(label)
            label = label[0]
            c1 = label[1:4]
            c2 = label[4:7]
            c3 = label[7:]

            c1 = 1 if sum(c1)>1 else 0
            c2 = 1 if sum(c2)>1 else 0
            c3 = 1 if sum(c3)>1 else 0

            # print(c1, c2, c3)

            image_name = filename +'_'+str(frame_counter)
            label_dict[image_name] = [c1, c2, c3]

            # print(label_dict)

            # Save the frame with corresponding labels
        
            output_file = os.path.join(output_dir_train, f'{image_name}.jpg')

            # Save the frame
            cv2.imwrite(output_file, frame)

        # Increment the frame counter
        frame_counter += 1

        # Break if we've reached the end of the video
        if frame_counter >= total_frames:
            break

with open('data_train.json', 'w') as j:
    json.dump(label_dict, j)

# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()




for filename in tqdm(val_list):

    # Set the path to the video file and CSV file
    filename = filename.split('.')[0]
    video_path = os.path.join(train_root, f'videos/{filename}.mp4')
    csv_path = os.path.join(train_root, f'labels/{filename}/frame.csv')

    # Open the video capture
    cap = cv2.VideoCapture(video_path)

    # Get the video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Read the CSV file into a pandas DataFrame
    labels_df = pd.read_csv(csv_path)

    # Initialize the frame counter
    frame_counter = 0


    # # Loop through the frames
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Check if the frame was read successfully
        if not ret:
            break

        # Get the labels for the current frame
        # print(frame_counter)
        if (frame_counter==0 or frame_counter%150 == 0):
            # print(frame_counter)
            label = labels_df.loc[labels_df['frame_id'] == frame_counter].values
            # print(label)
            label = label[0]
            c1 = label[1:4]
            c2 = label[4:7]
            c3 = label[7:]

            c1 = 1 if sum(c1)>1 else 0
            c2 = 1 if sum(c2)>1 else 0
            c3 = 1 if sum(c3)>1 else 0

            # print(c1, c2, c3)

            image_name = filename +'_'+str(frame_counter)
            label_dict[image_name] = [c1, c2, c3]

            # print(label_dict)

            # Save the frame with corresponding labels
        
            output_file = os.path.join(output_dir_val, f'{image_name}.jpg')

            # Save the frame
            cv2.imwrite(output_file, frame)

        # Increment the frame counter
        frame_counter += 1

        # Break if we've reached the end of the video
        if frame_counter >= total_frames:
            break

with open('data_val.json', 'w') as j:
    json.dump(label_dict, j)

# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()

100%|██████████| 70/70 [02:38<00:00,  2.27s/it]


In [ ]:
print(val_list[75:83])

In [62]:
for i, line in enumerate(train_list):
    if line == "":
        empty = i
        print(i, "empty")
    
# train_list = train_list[:empty] + train_list[empty+1:]

In [63]:
with open('val_list.txt', 'w') as f:
    for line in val_list:
        f.write(f"{line}\n")
with open('train_list.txt', 'w') as f:
    for line in train_list:
        f.write(f"{line}\n")

# Dataloader

In [46]:
import torch
import numpy as numpy
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
import pickle
from PIL import Image


class SingleImageDataset(Dataset):
    def __init__(self, pkl_file, transform=None):

        self.transform = transform
        if self.transform is None:
            transforms = v2.Compose([
                v2.Resize(224),
                v2.ToImage(),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        with open(pkl_file, 'rb') as f:
            self.data = pickle.load(f)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        item = self.data[idx]
        image = Image.fromarray(item['img_array'])
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(item['label'], dtype=torch.float32)
        return image, label


In [47]:
import torchvision
from torch.optim import AdamW
import torch.nn as nn
from torchvision.transforms import v2
import time

In [78]:
import torch
from sklearn.metrics import average_precision_score
import numpy as np
from tqdm import tqdm
def evaluate_map(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch_idx, (images, labels) in (enumerate(dataloader)):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            preds = torch.sigmoid(outputs)  # Sigmoid to get probabilities
            
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)

    # Calculate average precision for each label
    ap_per_label = []
    for i in range(all_labels.shape[1]):
        ap = average_precision_score(all_labels[:, i], all_preds[:, i])
        ap_per_label.append(ap)
    map_score = np.mean(ap_per_label)
    return map_score, ap_per_label

In [86]:
transforms = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

all_data = SingleImageDataset('single_images.pkl', transform=transforms)
train_set, val_set = torch.utils.data.random_split(all_data, [0.8, 0.2])



train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
torch.set_float32_matmul_precision('high')
device = 'cuda:0'

model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
model.fc = nn.Sequential(nn.Linear(2048, 3, bias=True))
model = model.to(device)
# model = torch.compile(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()
epochs = 50

for epoch in range(epochs):
    loss = 0
    start = time.time()

    for step, (datas, labels) in (enumerate(train_loader)):
        datas = datas.to(device)
        labels = labels.to(device)
        output = model(datas)

        loss = criterion(output, labels)
        loss.backward()
        norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        loss += loss.item()
        # print(f'Epoch: {epoch} | Step: {step} | Time: {round((end-start)*1000)} | Loss: {loss.item()}')
    end = time.time()
    print(f"Epoch {epoch} | validation mAP: {evaluate_map(model, val_loader, 'cuda:0')[0]} | avg_loss: {loss/(step+1)}")


23it [00:02, 10.60it/s]


Epoch 0 | validation mAP: 0.45641098051272583 | avg_loss: 0.00897589698433876


23it [00:02, 10.12it/s]


Epoch 1 | validation mAP: 0.3459917793736347 | avg_loss: 0.006703169085085392


23it [00:02, 10.37it/s]


...

Epoch 25 | validation mAP: 0.7121334285298225 | avg_loss: 0.005750473588705063


23it [00:02, 10.97it/s]


Epoch 26 | validation mAP: 0.7261451911312014 | avg_loss: 0.007490485906600952


23it [00:02, 10.78it/s]


Epoch 27 | validation mAP: 0.7355117818918213 | avg_loss: 0.0076883211731910706


In [76]:
evaluate_map(model, val_loader, 'cuda:0')

113it [00:11,  9.84it/s]


(0.22480193480979463,
 [0.16909663521797264, 0.3269857268493618, 0.1783234423620495])